In [3]:
import numpy as np
import cv2
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
pwd

'C:\\Users\\Dell\\face recognizer'

In [5]:
face_classifier = cv2.CascadeClassifier("haarcascade_frontalface_alt2.xml")
eye_classifier = cv2.CascadeClassifier('haarcascade_eye.xml')
smile_classifier = cv2.CascadeClassifier('haarcascade_smile.xml')

#Can use any of below three but the FischerFace is better

recognizer = cv2.face.LBPHFaceRecognizer_create()
#recognizer = cv2.face.EigenFaceRecognizer_create()
#recognizer = cv2.face.createFisherFaceRecognizer()

recognizer.read("trainner.yml")

In [6]:
#This piece of code for labels as output
labels = {"person_name":1}
with open("labels.pickle", 'rb') as f:
    real_labels = pickle.load(f)
    labels = {v:k for k,v in real_labels.items()}

In [7]:
cap = cv2.VideoCapture(0)
while(True):
    ret,frame = cap.read()
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
        #print(x,y,w,h)
        #(ycord_start, ycord_end)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        id_, conf = recognizer.predict(roi_gray)
        if conf>=45:
            #for printing only the id in output
            #print(id_)
            #for printing labels in output
            #print(labels[id_])
            #for putting text over the bounding box
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255, 255, 255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font, 1, color, stroke, cv2.LINE_AA)
            
            
        image_item = "my_image.png"
        cv2.imwrite(image_item,roi_gray)
        
        color = (255, 0, 0) #BGR 0-255 
        stroke = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x, y), (end_cord_x, end_cord_y), color, stroke)
        """eyes = eye_calssifier.detectMultiScale(roi_gray)
        for (ex,ey,ew,eh) in eyes:
            cv2.rectangle(roi_color,(ex,ey),(ex+ew,ey+eh),(0,255,0),2)"""
        
    cv2.imshow("frame",frame)
    if cv2.waitKey(20) &  0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()